In [8]:
pip show azureml-sdk

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install azureml-sdk==1.48.0

ERROR: Could not find a version that satisfies the requirement azureml-sdk==1.48.0 (from versions: 0.1.57, 0.1.58, 0.1.59, 0.1.65, 0.1.68, 0.1.74, 0.1.80, 1.0rc83, 1.0rc85, 1.0.2, 1.0.6, 1.0.8, 1.0.10, 1.0.15, 1.0.15.1, 1.0.17, 1.0.18, 1.0.18.1, 1.0.21, 1.0.23, 1.0.30, 1.0.33, 1.0.39, 1.0.41, 1.0.43, 1.0.45, 1.0.48, 1.0.53, 1.0.55, 1.0.57, 1.0.60, 1.0.62, 1.0.65, 1.0.69, 1.0.72, 1.0.74, 1.0.76, 1.0.79, 1.0.81, 1.0.83, 1.0.85, 1.1.0rc0, 1.1.1rc0, 1.1.2rc0, 1.1.5, 1.1.5.1, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6.0, 1.7.0, 1.7.0.post1, 1.8.0, 1.9.0, 1.10.0, 1.11.0, 1.12.0, 1.13.0, 1.14.0, 1.15.0, 1.16.0, 1.17.0, 1.18.0, 1.19.0, 1.20.0, 1.21.0, 1.22.0, 1.23.0, 1.24.0, 1.25.0, 1.49.0, 1.50.0, 1.51.0, 1.52.0, 1.53.0, 1.54.0, 1.55.0, 1.56.0, 1.57.0, 1.58.0, 1.59.0, 1.60.0)
ERROR: No matching distribution found for azureml-sdk==1.48.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-279907
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-279907


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_nodes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
pip install 'azureml-sdk[notebooks]'

  Using cached azureml_sdk-1.60.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached azureml_train_core-1.60.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached azureml_train_automl_client-1.60.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached azureml_pipeline-1.60.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached azureml_widgets-1.60.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached azureml_contrib_notebook-1.60.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached azureml_pipeline_core-1.60.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached papermill-2.6.0-py3-none-any.whl.metadata (13 kB)
  Preparing metadata (setup.py) ... - done
  Using cached azureml_pipeline_steps-1.60.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached azureml_automl_core-1.60.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached azureml_train_restclients_hyperdrive-1.60.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached ipywidgets-7.8.5-py

In [12]:
#from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train_wv.py',
                compute_target='Udacity',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [18]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from collections.abc import Mapping
from azureml.widgets import RunDetails


hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8557a51d-3964-4282-ba61-77a1000e76e9
Web View: https://ml.azure.com/runs/HD_8557a51d-3964-4282-ba61-77a1000e76e9?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-279907/workspaces/quick-starts-ws-279907&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-05-05T17:04:42.1154614Z][GENERATOR][WARNING]Space size : 12 is less than max total jobs : 100, only 12 jobs will be generated 
[2025-05-05T17:04:43.5452941Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-05-05T17:04:43.7218287Z][SCHEDULER][INFO]Scheduling job, id='HD_8557a51d-3964-4282-ba61-77a1000e76e9_0' 
[2025-05-05T17:04:43.7754417Z][SCHEDULER][INFO]Scheduling job, id='HD_8557a51d-3964-4282-ba61-77a1000e76e9_1' 
[2025-05-05T17:04:43.8775787Z][SCHEDULER][INFO]Scheduling job, id='HD_8557a51d-3964-4282-ba61-77a1000e76e9_3' 
[2025-05-05T17:04:43.8784095Z][SCHEDULER][INFO]Scheduling job, id='HD_8557a51d-3964-4282-ba61-77a1000e76e9_2' 
[2025-05

{'runId': 'HD_8557a51d-3964-4282-ba61-77a1000e76e9',
 'target': 'Udacity',
 'status': 'Completed',
 'startTimeUtc': '2025-05-05T17:04:41.754487Z',
 'endTimeUtc': '2025-05-05T17:13:47.434151Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'ba485349-bca0-4702-9702-8fd0eade74e9',
  'user_agent': 'python/3.10.16 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_8557a51d-3964-4282-ba61-77a1000e76e9_5',
  'score': '0.9146637533381888',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_8557a51d-3964-4282-ba61-77a1000e76e9_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train'

In [17]:
#from collections.abc import Mapping
#from azureml.widgets import RunDetails

#RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [19]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_8557a51d-3964-4282-ba61-77a1000e76e9_5,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-279907', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-279907'), name=best_hd_model.pkl, id=best_hd_model.pkl:2, version=2, tags={}, properties={})

In [20]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.1', '--max_iter', '100']


In [22]:
print(best_run.get_metrics())

{'Regularization Strength:': 0.1, 'Max iterations:': 100, 'Accuracy': 0.9146637533381888}


In [23]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [24]:

from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [14]:
# Submit your automl run

#experiment = Experiment(ws, 'automl')
#run = experiment.submit(config=automl_config, show_out=True)

# Submit your automl run
from azureml.core.experiment import Experiment

experiment = Experiment(ws, 'project1automl')
automl_run = experiment.submit(config=automl_config, show_out=True)

2025-05-05:16:09:24,346 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
project1automl,AutoML_9877f8fc-1fc9-4ed9-ad26-43d86246e236,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2025/05/05 16:14:33 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025-05-05:16:39:43,19 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-05:16:39:44,80 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-05:16:39:44,81 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-05:16:39:45,251 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-05:16:

In [37]:
# Retrieve and save your best automl model.
from azureml.core.experiment import Experiment

best_automl_run, fitted_automl_model = run.get_output()
best_automl_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_automl_run)

AttributeError: 'Run' object has no attribute 'get_output'

In [ ]:
print(best_auto_run.get_details())

In [ ]:
print(best_auto_run.get_metrics())

In [ ]:
cluster.delete()